# Importy:

In [187]:
# import os
# import json
# import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
# sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
# import torchvision
# from torchvision.utils import save_image
# from torchvision.datasets import FashionMNIST
# from torchvision.transforms import v2
# from torchvision.datasets import ImageFolder
# from torch.nn.functional import one_hot

# from pytorch_fid import fid_score

# from collections import Counter
from sklearn.model_selection import train_test_split

/tmp/ipykernel_7973/294544994.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [188]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cpu


In [189]:
import transformers
from transformers import *
from datasets import load_dataset
import evaluate
from tqdm import tqdm

# Wczytywanie danych

In [190]:
PATH = '../../data'
reviews = pd.read_csv(f"train_data.csv")
reviews.head()

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3


In [191]:
reviews['rating'].value_counts()

rating
4    7243
3    4831
2    1747
1    1434
0    1137
Name: count, dtype: int64

Z komórki wyżej wynika, że dane treningowe są niezbilansowane. \
Znamy 2 główne metody "naprawy" tego problemu: undersampling i oversampling. \
Zdecydowaliśmy się zastosować metodę undersamplingu, w nadziei, że prawie 1200 wierszy dla każdej z klas wystarczy.

In [192]:
X = reviews.filter(['review'])
y = reviews.filter(['rating'])

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [194]:
from imblearn.under_sampling import RandomUnderSampler

In [195]:
sampler = RandomUnderSampler(sampling_strategy="not minority")
resampled_train_data, resampled_train_classes = sampler.fit_resample(X_train, y_train)
resampled_test_data, resampled_test_classes = sampler.fit_resample(X_test, y_test)

In [196]:
resampled_train_classes.value_counts()

rating
0         790
1         790
2         790
3         790
4         790
Name: count, dtype: int64

In [197]:
resampled_test_classes.value_counts()

rating
0         347
1         347
2         347
3         347
4         347
Name: count, dtype: int64

TODO

Łącznie, resampled_train_classes oraz resampled_test_classes mają po 1137 w każdej z klas. \
\
Najpierw podzieliliśmy początkowy zbiór testowy na nowe dane testowe i walidacyjne, a następnie zbilansowaliśmy dane testowe otrzymane w wyniku podziału. Kolejność tych operacji jest ważna, ponieważ chcemy, aby zbiór walidacyjny nie zawierał żadnych zmian (aby zbiór walidacyjny był jak najbliższy "rzeczywistym" danym).

In [198]:
resampled = pd.merge(resampled_train_data, resampled_train_classes, left_index=True, right_index=True)
resampled_test = pd.merge(resampled_test_data, resampled_test_classes, left_index=True, right_index=True)

In [199]:
resampled.head()

,review,rating
4635,disappointed service there-the restaurant open...,0
13177,"run thought picking cheap hotel save lot, not,...",0
5778,"rating suspect, hotel sofitel june 6-7 power w...",0
3526,"words, ok stay riu hotel punta cana, twice sta...",0
982,n't fooled price location grade room damrak ho...,0


In [200]:
resampled['rating'].value_counts()

rating
0    790
1    790
2    790
3    790
4    790
Name: count, dtype: int64

# Modele

## Model Rekurencyjny

In [15]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)

In [16]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

In [17]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [18]:
lstm_model = LSTMRegressor(100, 100, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 100)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

## Bag of Words

In [201]:
from bs4 import BeautifulSoup
import re

Aby usprawnić działanie metody BoW, zastosujemy tzw. "stopwords".

In [202]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/michal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [203]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

"Data cleaning"

In [204]:
def review_to_words(raw_review):
    """Function to convert a review to a string of words.
    The input is a single string (a raw moviw review), and the output is a single string (a preprocessed movie review)"""
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))
    meaningful_words = [word for word in words if not word in stops]
    return " ".join(meaningful_words)

In [219]:
def get_clean_reviews(reviews):
    clean_reviews = []
    num_reviews = reviews['review'].size

    for i, review in enumerate(reviews['review']):
        if i % 200 == 0:
            print('Review {} of {}'.format(i, num_reviews))
        clean_reviews.append(review_to_words(review))
    return clean_reviews

In [220]:
i = 0
clean_reviews_train = []
num_reviews = reviews['review'].size

clean_reviews_train = get_clean_reviews(resampled)
clean_reviews_test = get_clean_reviews(resampled_test)

Review 0 of 3950
Review 200 of 3950
Review 400 of 3950
Review 600 of 3950


/tmp/ipykernel_7973/3593606712.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'lxml').get_text()


Review 800 of 3950
Review 1000 of 3950
Review 1200 of 3950
Review 1400 of 3950
Review 1600 of 3950
Review 1800 of 3950
Review 2000 of 3950
Review 2200 of 3950
Review 2400 of 3950
Review 2600 of 3950
Review 2800 of 3950
Review 3000 of 3950
Review 3200 of 3950
Review 3400 of 3950
Review 3600 of 3950
Review 3800 of 3950
Review 0 of 1735
Review 200 of 1735
Review 400 of 1735
Review 600 of 1735
Review 800 of 1735
Review 1000 of 1735
Review 1200 of 1735
Review 1400 of 1735
Review 1600 of 1735


In [221]:
print('Creating the bag of words...')
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 3000)

train_data_features = vectorizer.fit_transform(clean_reviews_train)
valid_data_features = vectorizer.fit_transform(clean_reviews_test)

train_data_features = train_data_features.toarray()
valid_data_features = valid_data_features.toarray()
print('Bag of words completed')

Creating the bag of words...
Bag of words completed


In [222]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))

3000


In [223]:
flatten_train_classes = resampled_train_classes.to_numpy().flatten()
flatten_test_classes = resampled_test_classes.to_numpy().flatten()

In [224]:
train_indices = np.random.rand(len(reviews))>0.3
train_data = torch.from_numpy(train_data_features).float()
train_targets = torch.from_numpy(resampled['rating'].values).long()

test_data = torch.from_numpy(valid_data_features).float()
test_targets = torch.from_numpy(resampled_test['rating'].values).long()

In [28]:
# train_data = torch.from_numpy(train_data_features).float()
# train_targets = torch.from_numpy(flatten_train_classes).long()

# test_data =

In [225]:
train_dataset = torch.utils.data.TensorDataset(train_data, train_targets)
test_dataset = torch.utils.data.TensorDataset(test_data, test_targets)

In [226]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [231]:
class BoWClassifier(nn.Module):
    def __init__(self):
        super(BoWClassifier, self).__init__()
        self.lin1 =nn.Linear(3000, 512)
        self.d1 = nn.Dropout1d(0.3)
        self.bn1 = nn.BatchNorm1d(512)
        self.act1 =nn.LeakyReLU()

        self.lin2 =nn.Linear(512, 256)
        self.d2 = nn.Dropout1d(0.3)
        self.bn2 = nn.BatchNorm1d(256)
        self.act2 =nn.LeakyReLU()

        self.lin3 = nn.Linear(256, 124)
        self.d3 = nn.Dropout1d(0.3)
        self.bn3 = nn.BatchNorm1d(124)
        self.act3 = nn.LeakyReLU()

        self.lin4 = nn.Linear(124, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.act4 = nn.LeakyReLU()
        self.d4 = nn.Dropout1d(0.3)

        self.act5 = nn.LeakyReLU()
        self.lin5 =nn.Linear(64, 5)


    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.bn1(x)
        x = self.d1(x)

        x = self.lin2(x)
        x = self.act2(x)
        x = self.bn2(x)
        x = self.d2(x)

        x = self.lin3(x)
        x = self.act3(x)
        x = self.bn3(x)
        x = self.d3(x)

        x = self.lin4(x)
        x = self.act4(x)
        x = self.bn4(x)
        x = self.d4(x)

        x = self.lin5(x)
        return x
bow_model = BoWClassifier().to(device)
bow_model

BoWClassifier(
  (lin1): Linear(in_features=3000, out_features=512, bias=True)
  (d1): Dropout1d(p=0.3, inplace=False)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (d2): Dropout1d(p=0.3, inplace=False)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=256, out_features=124, bias=True)
  (d3): Dropout1d(p=0.3, inplace=False)
  (bn3): BatchNorm1d(124, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (lin4): Linear(in_features=124, out_features=64, bias=True)
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act4): LeakyReLU(negative_slope=0.01)
  (d4): Dropout1d(p=0.3, inplace=False)
  (act5): LeakyReLU(negative_slope=0.01)
  (lin5):

In [232]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for x, labels in data_loader:
        x, labels = x.to(device), labels.to(device)
        output = model(x)
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += x.shape[0]
    return correct / total

In [235]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bow_model.parameters(), lr=0.001)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(100):
    epoch_losses = []
    for x, labels in iter(train_loader):
        x, labels = x.to(device), labels.to(device)
        bow_model.train()
        out = bow_model(x).squeeze()

        loss = criterion(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(bow_model, test_loader)
    train_acc.append(get_accuracy(bow_model, train_loader)) # compute training accuracy
    val_acc.append(test_acc)  # compute validation accuracy
    print(f"Epoch {n} loss {loss_mean:.3} train_acc: {train_acc[-1]:.3} valid_acc {test_acc}")


print("Final Training Accuracy: {}".format(train_acc[-1]))
print("Final Validation Accuracy: {}".format(val_acc[-1]))

Epoch 0 loss 1.26 train_acc: 0.974 valid_acc 0.20576368876080692
Epoch 1 loss 1.25 train_acc: 0.977 valid_acc 0.21037463976945245
Epoch 2 loss 1.25 train_acc: 0.977 valid_acc 0.19711815561959653
Epoch 3 loss 1.25 train_acc: 0.984 valid_acc 0.19827089337175793
Epoch 4 loss 1.25 train_acc: 0.985 valid_acc 0.2
Epoch 5 loss 1.25 train_acc: 0.985 valid_acc 0.20576368876080692
Epoch 6 loss 1.25 train_acc: 0.987 valid_acc 0.20403458213256484
Epoch 7 loss 1.25 train_acc: 0.986 valid_acc 0.20576368876080692
Epoch 8 loss 1.24 train_acc: 0.987 valid_acc 0.21037463976945245
Epoch 9 loss 1.25 train_acc: 0.988 valid_acc 0.21210374639769453
Epoch 10 loss 1.25 train_acc: 0.989 valid_acc 0.21440922190201728
Epoch 11 loss 1.24 train_acc: 0.991 valid_acc 0.22420749279538904
Epoch 12 loss 1.24 train_acc: 0.99 valid_acc 0.2201729106628242
Epoch 13 loss 1.25 train_acc: 0.991 valid_acc 0.2132564841498559
Epoch 14 loss 1.25 train_acc: 0.994 valid_acc 0.2195965417867435
Epoch 15 loss 1.25 train_acc: 0.992 vali

KeyboardInterrupt: 

## Gotowe modele z biblioteki sklearn

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import FunctionTransformer

In [35]:
pipelineMultinomialNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('MNB', MultinomialNB())
])

pipelineComplementNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('CNB', ComplementNB())
])

pipelineLinearSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('LSVC', LinearSVC())
])

pipelineSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('SVC', SVC())
])

In [36]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))

In [37]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))
# print(resampled_train_data.shape)
# print(resampled_train_classes.shape)
# print(resampled_train_data.values.reshape(-1))
# print(resampled_train_classes.values.reshape(-1))
# print(resampled_train_data.values.shape)
# print(resampled_train_classes.values.reshape(-1,1).shape)

In [38]:
pipelineMultinomialNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictMNB = pipelineMultinomialNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictMNB))

0.5079300528670191


In [39]:
pipelineComplementNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictCNB = pipelineComplementNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictCNB))

0.5453436356242375


In [40]:
pipelineLinearSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictLSVM = pipelineLinearSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictLSVM))

0.5461569743798292


In [41]:
pipelineSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictSVM = pipelineSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictSVM))

0.5673037820252135


Sposród przetestowanych przez nas gotowych modeli dostarczonych w bibliotece scikit-learn, najlepsze wyniki daje model SVM. Zauważyliśmy przy tym, że przeprowadzenie całości ostatniego pipeline'u zajmuje znacznie więcej czasu (~25-30 sek.) w porównaniu do pozostałych trzech modeli (~0.7 sek.).

## Modele rekurencyjne

In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)


In [ ]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [ ]:
tokenizer["aa"]

5636

In [ ]:
clean_train_reviews_tokenized = []
for review in reviews['review']:
    unknows = 0
    all_parsed = 0
    review_tokenized = []
    for word in review.split():
        all_parsed+=1
        try:
            review_tokenized.append(tokenizer[word.lower()])
        except:
            unknows +=1
    clean_train_reviews_tokenized.append(review_tokenized)

In [ ]:
clean_train_reviews_tokenized

In [ ]:
class ReviewDataset(data.Dataset):
    def __init__(self, data,labels):
        self.data = []
        for d, l in zip(data,labels):
            self.data.append((torch.from_numpy(np.array(d)).long(),torch.tensor(l).long()))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [ ]:
train_indices = np.random.rand(len(clean_train_reviews_tokenized))>0.3

array([ True,  True,  True, ..., False,  True, False])

In [ ]:
train_data = ReviewDataset(np.array(clean_train_reviews_tokenized, dtype=object)[train_indices],reviews["rating"].values[train_indices])
test_data = ReviewDataset(np.array(clean_train_reviews_tokenized, dtype=object)[~train_indices],reviews["rating"].values[~train_indices])

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x)-1 for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=1)
    yy = torch.stack(yy)
    return xx_pad, yy, x_lens

In [ ]:
train_loader = data.DataLoader(train_data, batch_size=32, collate_fn=pad_collate, shuffle=True,drop_last=True)
test_loader = data.DataLoader(test_data, batch_size=32, collate_fn=pad_collate, shuffle=False)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

lstm_model = LSTMRegressor(100, 100, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 100)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss()
lstm_model.train()

# Training loop
for epoch in range(31):
    losses = 0
    batches = 0
    for x, targets, len_x in train_loader:
        x = x.to(device)
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        batches +=1
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {losses/batches:.3}")

In [ ]:
lstm_model.eval()
with torch.no_grad():
    preds_list = []
    targets_list = []
    for x, targets, len_x in test_loader:
        x = x.to(device)
        targets_list.append(targets.numpy())
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        preds_list.append(preds.cpu().numpy())

In [ ]:
print(f"Test accuracy: {(np.argmax((np.concatenate(preds_list)),1) == np.concatenate(targets_list)).sum()/len(np.concatenate(targets_list)):.3}")

Test accuracy: 0.539


In [ ]:
example_1_text = "I do not like this hotel"
example_2_text = "I like this hotel"
example_1_tokenized = []
for word in example_1_text.split():
    try:
        example_1_tokenized.append(tokenizer[word])
    except:
        continue
example_2_tokenized = []
for word in example_2_text.split():
    try:
        example_2_tokenized.append(tokenizer[word])
    except:
        continue
hidden, state = lstm_model.init_hidden(1)
hidden, state = hidden.to(device), state.to(device)
preds_1,_ = lstm_model(torch.from_numpy(np.array(example_1_tokenized)).unsqueeze(0).to(device),len(example_1_tokenized)-1,(hidden,state))
preds_2,_ = lstm_model(torch.from_numpy(np.array(example_2_tokenized)).unsqueeze(0).to(device),len(example_2_tokenized)-1,(hidden,state))

In [ ]:
print(np.argmax(preds_1.detach().numpy()))
print(np.argmax(preds_2.detach().numpy()))

3
3


## Embbedings with bert

In [ ]:
class ReviewDataset(data.Dataset):
    def __init__(self, data, mask, labels):
        self.data = data
        self.mask = mask
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data = self.data[idx]
        in_mask = self.mask[idx]
        in_target = self.labels[idx]
        return in_data, in_mask, in_target

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
inputs_two = tokenizer(reviews['review'].to_numpy().tolist(), return_tensors='pt', padding=True, truncation=True)

In [ ]:
inputs = inputs_two['input_ids']
attention_mask = inputs_two['attention_mask']
train_indices = np.random.rand(len(inputs))>0.3

In [ ]:
train_data = ReviewDataset(inputs[train_indices], attention_mask[train_indices], reviews["rating"].values[train_indices])
test_data = ReviewDataset(inputs[~train_indices], attention_mask[~train_indices], reviews["rating"].values[~train_indices])

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x)-1 for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=1)
    yy = torch.stack(yy)
    return xx_pad, yy, x_lens

In [ ]:
train_loader = data.DataLoader(train_data, batch_size=128, shuffle=True,drop_last=True)
test_loader = data.DataLoader(test_data, batch_size=128,  shuffle=False)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bert, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.bert = bert
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, attention_mask, hidden):
        x = self.bert(input_ids=x, attention_mask=attention_mask)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

lstm_model = LSTMRegressor(100, 100, 1, 5, bert=model).to(device)
lstm_model

LSTMRegressor(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [ ]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss()
lstm_model.train()

# Training loop
for epoch in range(31):
    losses = 0
    batches = 0
    for x, mask, targets in train_loader:
        x = x.to(device)
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, mask, (hidden, state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        batches +=1
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {losses/batches:.3}")

: 

In [ ]:
lstm_model.eval()
with torch.no_grad():
    preds_list = []
    targets_list = []
    for x, targets, len_x in test_loader:
        x = x.to(device)
        targets_list.append(targets.numpy())
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        preds_list.append(preds.cpu().numpy())

In [ ]:
print(f"Test accuracy: {(np.argmax((np.concatenate(preds_list)),1) == np.concatenate(targets_list)).sum()/len(np.concatenate(targets_list)):.3}")

Test accuracy: 0.539


In [ ]:
example_1_text = "I do not like this hotel"
example_2_text = "I like this hotel"
example_1_tokenized = []
for word in example_1_text.split():
    try:
        example_1_tokenized.append(tokenizer[word])
    except:
        continue
example_2_tokenized = []
for word in example_2_text.split():
    try:
        example_2_tokenized.append(tokenizer[word])
    except:
        continue
hidden, state = lstm_model.init_hidden(1)
hidden, state = hidden.to(device), state.to(device)
preds_1,_ = lstm_model(torch.from_numpy(np.array(example_1_tokenized)).unsqueeze(0).to(device),len(example_1_tokenized)-1,(hidden,state))
preds_2,_ = lstm_model(torch.from_numpy(np.array(example_2_tokenized)).unsqueeze(0).to(device),len(example_2_tokenized)-1,(hidden,state))

In [ ]:
print(np.argmax(preds_1.detach().numpy()))
print(np.argmax(preds_2.detach().numpy()))

3
3
